## Fluídos Conformes

Código para testear la estabilidad alrededor de una solución de equilibrio.

Las ecuaciones son:

$(\zeta^a, \zeta^{ab})$ -> $f^j = (\mu, v, x_i)$ -> $(-1,0,0,0,0)$

$$
\partial_t c^j = \partial_i F^{ij}(f) + I^j(f) = \frac{\partial F^{ij}}{\partial f^k}\partial_i f^k
$$

$$
c^j = C^j(f) = (e, S, c_i)
$$

Lo que hacemos es tomar el jacobiano del flujo,

$\frac{\partial F^i}{\partial f^j}$ donde $f^j$ son las variables no conservativas para las cuales tenemos las expresiones del Flujo. 
A estas las multiplicamos por $[\frac{\partial c^k}{\partial f^j}]^{-1}$.
Tenemos así 

$$
P^{il}{}_j := \frac{\partial F^{il}}{\partial c^j} = \frac{\partial F^{il}}{\partial f^k}\frac{\partial f^k}{\partial c^j}.
$$

Evaluamos todo en el vector $(-1,0,0,0,0)$ o sea estamos en el frame con velocidad cero y todas las variables fuera de equilibrio también cero.

Lo mismo hacemos con la fuente. 

$$
II^i{}_j = \frac{\partial I^i}{\partial f^k} \frac{\partial f^k}{\partial c^j}
$$

Tenemos así el sistema linearizado en el punto de equilibrio en las variables conservativas. Multiplicamos $P^i{}_j$ por un número complejo cualquiera y para todos ellos los autovalores debieran tener parte real negativa o nula. Pero no es así...


Alternativamente podemos diferenciar $c^i$ con respecto a los $f^j$ para obtener,

$$
\partial_t c^j = \frac{\partial c^j}{\partial f^i} \partial_t f^i
$$

y por lo tanto,

$$
\partial_t f^i = \frac{\partial c^j}{\partial f^i}^{-1} \frac{\partial F^{lj}}{\partial f^k}\partial_l f^k := \tilde{P}^i{}_k\partial_l f^k
$$

A las que simplemente sumamos el término $\frac{\partial c^j}{\partial f^i}^{-1}\frac{\partial I^j}{\partial f^k}$ para tener el sistema linearizado con respecto a las $f^i$.

In [196]:
using Symbolics
using LinearAlgebra
using Latexify

In [256]:
function Flux(flu,par)
    χ = par
    #flux = zeros(5)
    μ = flu[1]  
    T = (abs(μ))^(-1//2) 
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    χ₀ = χ[1]
    χ₁ = χ[2]
    χ₂ = χ[3]
    γ = (1 - v^2)^(-1//2)
    τ = 2χ₁ * x3 * T / (γ*μ^3) + 24χ₂*(1//2*(1-v^2)x3^2 + 14//3 * x2^2 + 7/5*(1-v^2)x1*x3)/μ^5
    ρ = -6χ₀ / μ^2 - 6χ₁*x1/(γ * μ^4 * T) + 42χ₂*(6//5 *x1^2 + 10γ^2*x2^2 + 3//2*(v^2-1)^2*x3^2)/(μ^5 * γ^2)
#    Q = 10χ₀ * x2 * T / μ^3 + 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)
    Q = 10χ₁ * x2 * T / μ^3 + 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)
    #flux[1] = 4//3 * ρ*γ^2*v + γ*Q*(1+v^2)+ τ*v
    #flux[2] = 4//3 * ρ*(γ^2*v^2 + 1//4) + 2v*γ*Q + τ
    #flux[3] = χ₁*γ*v*(6*γ^2 - 1)/μ^3/T - 12χ₂*(v*(6γ^2 - 1)*x1 + (6γ^2*(2v^2 + 1)-1)*x2 + v*(v^2+2)*x3)/μ^4
    #flux[4] = χ₁*γ*(6γ^2*v^2+1)/μ^3/T - 12χ₂*((6γ^2*v+1)*x1 + v*(6γ^2*(1+v^2)+1)*x2 + (2v^2+1)*x3)/μ^4
    #flux[5] = 3χ₁*γ*v*(2γ^2*v^2+1)/μ^3/T - 12χ₂*(v*(6γ^2*v^2+3)*x1 +3*(1+6γ^2*v^2)*x2 + 3v*x3)/μ^4
    #return -flux[:]
    return [4//3 * ρ*γ^2*v + γ*Q*(1+v^2)+ τ*v; 
            4//3 * ρ*(γ^2*v^2 + 1//4) + 2v*γ*Q + τ;
            χ₁*γ*v*(6*γ^2 - 1)/μ^3/T   - 12χ₂*(v*(6γ^2 - 1)*x1   + (6γ^2*(2v^2 + 1)-1)*x2 + v*(v^2+2)*x3)/μ^4;
            #χ₁*γ*(6γ^2*v^2+1)/μ^3/T    - 12χ₂*((6γ^2*v+1)*x1     + v*(6γ^2*(1+v^2)+1)*x2  + (2v^2+1)*x3)/μ^4;
            χ₁*γ*(6γ^2*v^2+1)/μ^3/T    - 12χ₂*((6γ^2*v^2+1)*x1     + v*(6γ^2*(1+v^2)+1)*x2  + (2v^2+1)*x3)/μ^4;
            3χ₁*γ*v*(2γ^2*v^2+1)/μ^3/T - 12χ₂*(v*(6γ^2*v^2+3)*x1 + 3*(1+6γ^2*v^2)*x2      + 3v*x3)/μ^4]
end

Flux (generic function with 1 method)

Chequeamos con los valores de Marcelo. 

In [257]:
χ = [- 1.; - 2.; - 10.]
flu = [- 5.; 0.5; 2.1; 0.5; 5.1]
Flu = [5.921317694643806; 6.02302807825841; 3.713391378258412;  4.136735467078638; 3.444079555898864]

Flux(flu,χ) - Flu;

Calculamos el Jacobiano

In [258]:
@variables f[1:5], c[1:5], p[1:3]#, fl[1:5]

JFS = Symbolics.jacobian(Flux(f,p),f);
JF_exp = Symbolics.build_function(JFS, f, p);
JFac = eval(JF_exp[1]);

Definimos algunos valores para testear

In [259]:
@variables x1, x2, x3

flu=[-1.;0;0;0;0];
con=ones(5)
χ=[-1.,-0.5,-5.]

3-element Vector{Float64}:
 -1.0
 -0.5
 -5.0

El Jacobiano en equilibrio.

In [260]:
JFac(flu,p)

5×5 Matrix{Num}:
  0.0    (-8//1)*p₁          0.0    -10.0p₂      0.0
 -4.0p₁           0.0       -2.0p₂    0.0       -2.0p₂
  0.0            -5.0p₂     -0.0    -60.0p₃     -0.0
 -2.5p₂           0.0    -12p₃       -0.0    -12p₃
  0.0            -3.0p₂     -0.0    -36.0p₃     -0.0

Ahora traemos el Jacobiano de la transformación entre las variables fluido/conservativas.

In [261]:
include("inversion_ext.jl");

In [262]:
@variables χ₂ μ γ v

A = (-12 * χ₂/μ^4)*[3(2γ^2 - 1)  3v*(6γ^2 - 1)    3v^2 ;
                v*(6γ^2 - 1)  (6γ^2*(1 + 2v^2) - 1)  v*(v^2 + 2) ;
                (6γ^2 * v^2 + 1)  v*(6γ^2*(2 + v^2) - 1)  (2v^2 + 1)]

A[1,2]


-36v*χ₂*(μ^-4)*(6(γ^2) - 1)

Para posterior uso calculamos las variables conservativas correspondientes al estado de equilibrio. Esto no me lo esperaba, pensaba que me darían c1=c2=c3=0!

In [263]:
con = F(flu,zeros(5), p);

Jac(flu,con,p)

5×5 Matrix{Num}:
 -12.0p₁           0.0       -6.0p₂    0.0        0.0
   0.0    (-8//1)*p₁          0.0    -10.0p₁      0.0
  -7.5p₂          -0.0      -36.0p₃   -0.0        0.0
   0.0           -15.0p₂     -0.0    -60.0p₃     -0.0
  -7.5p₂           0.0    -12p₃       -0.0    -12p₃

In [264]:
P(flu,c,p) = Jac(flu,c,p) \ JFac(flu,p)

P (generic function with 1 method)

In [265]:
flu = [-1;0.9;0;0;0]
M0=simplify.(substitute.(P(flu,c,p), (Dict(p[2] => 0),)))

5×5 Matrix{Num}:
  0.821918   0.913242  -8.57739e-18    2.30811e-14  -1.6297e-18
  0.024726   0.821918  -0.00100239     2.69735      -0.000190454
 -0.0       -0.0        0.910243      63.8535        0.00194622
 -0.0       -0.0        0.0096827    -26.0554        0.00183971
 -0.0       -0.0        0.291286     167.936         0.955344

In [266]:
P(flu,c,p);

In [267]:
18.75/3.75*36

180.0

In [268]:
P(flu,c,p)[4,3];

In [269]:
P(flu,c,p)';

In [270]:
Ratio = P(flu,c,p)'[4,1]/P(flu,c,p)'[4,3];

In [271]:
#par_f = [-1,-0.5,-10]
#substitute(P(flu,c,p)'[4,1]/P(flu,c,p)'[4,3], (Dict(p => par_f),))


In [272]:
simplify(12*(10p[1]*(-36p[3] - 3.75p[2]^2/p[1]) + 6p[2]*(6.25p[2] - 60p[3])) / (6.25p[2] - 60p[3]) / p[1])

(p₁^-1)*((6.25p₂ - (60p₃))^-1)*(120p₁*(-36p₃ - (3.75(p₁^-1)*(p₂^2))) + 72p₂*(6.25p₂ - (60p₃)))

In [273]:
par_f = [-1,-0.5,-10]
P(flu,c,par_f)'[4,1]/P(flu,c,par_f)'[4,3]

-0.09313371727529571

Miramos primero el caso donde las variables están desacopladas, es decir con $\chi_1=0$.

In [274]:
P(flu,c,[-1,-0,-2])

5×5 Matrix{Float64}:
 0.821918  0.913242  -9.68895e-18    1.9843e-14  -1.73017e-18
 0.024726  0.821918  -0.00100239     2.69735     -0.000190454
 0.0       0.0        0.910243      63.8535       0.00194622
 0.0       0.0        0.0096827    -26.0554       0.00183971
 0.0       0.0        0.291286     167.936        0.955344

Vemos que se corresponde con la siguiente matriz racional.

In [275]:
M = [0 2//3 0 0 0;
    1//2 0 -1//4 0 -1//4;
    0 0 0 5//3 0;
    0 0 1//5 0 1//5;
    0 0 0 4//3 0];

In [276]:
P(flu,c,[-1,0,-1]) - M

5×5 Matrix{Float64}:
  0.821918  0.246575  -9.68895e-18    1.9843e-14  -1.73017e-18
 -0.475274  0.821918   0.248998       2.69735      0.24981
  0.0       0.0        0.910243      62.1869       0.00194622
  0.0       0.0       -0.190317     -26.0554      -0.19816
  0.0       0.0        0.291286     166.602        0.955344

In [277]:
eigvals(M)

5-element Vector{Float64}:
 -0.7745966692414842
 -0.5773502691896256
  3.0357660829594124e-18
  0.5773502691896258
  0.7745966692414837

In [278]:
1/0.7745966692414837^2 - 5/3

-1.5543122344752192e-15

In [279]:
1/0.5773502691896258^2 - 3

-8.881784197001252e-16

In [280]:
eigvals(P(flu,c,[-1,-0.5,-1]))

5-element Vector{ComplexF64}:
 0.6995181579170078 + 0.0im
 0.9000000000000455 + 0.0im
  0.966976201839825 - 0.08080914590484302im
  0.966976201839825 + 0.08080914590484302im
  28.35853816389217 + 0.0im

Hacemos lo mismo para la fuente.

In [281]:
function Is(flu, χ, ξ)
    #(χ, ξ) = par 
    μ = flu[1] 
    χ₀= χ[1]
    χ₁= -χ[2] # lo hacemos positivo
    κ = χ₀*ξ[1]/χ₁^2 # OK 
    λ = χ₀*ξ[2]/χ₁^2  # OK
    η = χ₀*ξ[3]/χ₁^2
    T = (abs(μ))^(-1//2) # use μ positive, so I changed μ -> -μ
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    γ = (1. - v^2)^(-1//2)
    #Is[1] = 0.
    #Is[2] = 0.
    #Is[3] = -2//5*a*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ)
    #Is[4] = -2//5*a*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ)
    #Is[5] = -2//5*a*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ)
    #return Is[:]      #(1 - ℯ^(-5. *t))
    return [0.;
            0.;
            -(3//10*(γ^2-1//4)*x1/(γ*λ)   + 2γ*v*x2/κ      - v^2*x3/(γ*η))/μ^4/T;
            -(3//10*γ*v*x1/λ              + γ*(v^2+1)*x2/κ - v*x3/(γ*η))/μ^4/T;
            -(3//10*(γ^2*v^2+1//4)*x1/λ/γ + 2γ*v*x2/κ      - x3/(γ*η))/μ^4/T;
            #-2//5*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*η);
            #-2//5*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*η);
            #-2//5*(γ^2*v^2+1//4)*T*x1/η/γ - 2γ*v*x2/T/κ + T*x3/(γ*η)
            #-2//5*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ);
            #-2//5*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ);
            #-2//5*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ)
            ]
end

Is (generic function with 1 method)

La chequeamos con valores de Marcelo.

In [282]:
t=0
flu = [- 5.; 0.0; 2.1; 0.5; 5.1]
par = [-1., -2., -10.]
par_s = [1/15π, 1/1π, 1/1π]
I_c = [0.; 0.; - 0.09488575328013785; - 0.12155655385334033; - 0.12140081195218293]
Is(flu,par,par_s) - I_c;

In [283]:
@variables q[1:3]  
Is(f,p,q);

In [284]:
JSS = Symbolics.jacobian(Is(f,p,q),f);

In [285]:
JS_exp = Symbolics.build_function(JSS, f, p,q);

In [286]:
JSac = eval(JS_exp[1]);

In [287]:
flu=[1.;-0.9;0;0;0];
con=ones(5)
#par_f = [-1,-2,-10]
par_f = [-1,-0.0,1]
#JSac(flu,par_f,par_s)
#eigvals(JSac(flu,par_f,par_s)) #esta no es la matríz correcta para sacarle los autovalores
eigvals(P(flu,c,par_f))

5-element Vector{Float64}:
 -0.9998591951085865
 -0.9721870563644237
 -0.9000000000000027
 -0.6716485600739354
 26.089634837175293

Si $\chi_1=0$ los autovalores (velocidades) no dependen de los otros $\chi$ y hay uno que crece muchísimo cuando $v \to 1$. Eso trae problemas para evolucionar las ecuaciones por el CFL.

In [288]:
II(flu,c,p,q) = Jac(flu,c,p) \ JSac(flu,p,q)

II (generic function with 1 method)

In [289]:
II(flu,c,p,q);

In [494]:
flu=[1.;0.0;0;0;0];
#con=ones(5)
par_f = [-1.,-1.,-10.]
#par_f = [-1,-0.5,-10]
par_s = [-1/15π, -1/1π, 0.01/1π]
con = F(flu,zeros(5), par_f);
eigvals(10*im .* P(flu,c,par_f) + 1.0 .*II(flu,c,par_f,par_s))

5-element Vector{ComplexF64}:
   -1.5305241965462313 - 8.782912303847589e-17im
   -0.5871196517497985 - 7.531146728810917im
   -0.5871196517497976 + 7.531146728810912im
 0.0018765050321101536 + 5.834652395235965im
  0.001876505032112119 - 5.834652395235972im

Vemos que un par de autovalores tiene parte real positiva. 

In [292]:
eigvecs(II(flu,c,par_f,par_s))

5×5 Matrix{Float64}:
  0.104016   1.0  0.0   0.107241     0.00723379
 -0.0124128  0.0  1.0   0.00259509  -0.00110536
  0.931304   0.0  0.0   0.990192    -0.00388408
 -0.112165   0.0  0.0   0.0230413    0.0127446
  0.330331   0.0  0.0  -0.0864993    0.999884

In [293]:
par_f = [-1,-1,-10]
E = eigvecs(10*im .* P(flu,c,par_f) - 0. * II(flu,c,par_f,par_s));

In [294]:
[E[:,1]; E[:,5]]

10-element Vector{ComplexF64}:
  -0.015344969888471274 + 0.003445323142514261im
 -0.0007980235887311218 + 0.0018075801829751932im
   -0.06032668146796063 + 0.026905134456165127im
 -0.0009403840054966175 + 0.000796080530109239im
     0.9976893527729312 + 0.0im
  -0.015344969888471139 - 0.0034453231425145875im
 -0.0007980235887311539 - 0.0018075801829751602im
   -0.06032668146796097 - 0.026905134456164416im
 -0.0009403840054966171 - 0.0007960805301092484im
     0.9976893527729311 + 0.0im

In [295]:
[E[:,2]; E[:,4]]

10-element Vector{ComplexF64}:
  -0.11090389922227999 - 7.849623728795052e-17im
  -0.02205647919831648 + 4.130810277169772e-17im
   -0.5641482184214212 + 9.107298248878237e-17im
  -0.17599469206449075 - 2.862293735361732e-17im
    0.7987343068128037 + 0.0im
   0.11550044927507332 - 2.2967738821932926e-15im
 -0.011046546883667002 + 2.454633718507182e-16im
    0.8880023656553209 + 0.0im
  0.001699236800171895 - 2.6129272356900657e-17im
   0.44495677453439636 + 1.7261192475359621e-13im

In [296]:
E[:,3]

5-element Vector{ComplexF64}:
   0.9019719596318727 + 0.0im
 -0.14525761704623055 + 8.951173136040325e-16im
  0.04817527013142562 - 3.731043252130917e-14im
 0.000458579687273411 + 3.0327845269850773e-16im
 -0.40376446324599236 + 2.7622348852673895e-13im

In [297]:
Ei = inv(E)

5×5 Matrix{ComplexF64}:
 0.598753-41.506im        2.24604-254.961im      …     0.496231-0.648916im
 0.350127+2.22045e-16im   2.14983+5.34989e-15im     0.000861669+3.91959e-18im
  1.14615-1.96964e-14im  0.158511-6.61931e-14im        0.008423+7.12934e-17im
 -2.27353+8.49194e-14im  -13.7875+1.71599e-13im       0.0281913-1.61378e-16im
 0.598753+41.506im        2.24604+254.961im            0.496231+0.648916im

In [298]:
Ei[3,:]

5-element Vector{ComplexF64}:
   1.1461517267733188 - 1.9696375606892413e-14im
   0.1585112758557304 - 6.619314471545401e-14im
 -0.15090476395923247 + 2.9575408962145833e-15im
 -0.22016861243535946 + 1.1038045477640424e-14im
 0.008422995374060258 + 7.129340990304362e-17im

In [299]:
Ei[5,:]'*E[:,5]

-6.439293542825908e-15 + 1.4307131868118717e-15im

In [300]:
ET = eigvecs(10*im .* P(flu,c,par_f)');

In [301]:
[ET[:,1]; ET[:,5]];

In [302]:
[ET[:,2]; ET[:,3]];

In [303]:
ET[:,4] - ET[1,4]/Ei[4,1] .* Ei[4,:];

In [304]:
1/(Ei[3,1]/Ei[3,3])

-0.1316621180548793 + 3.178238598760267e-16im

In [305]:
1/(Ei[3,1]/Ei[3,5])

0.007348935727534898 + 1.8849232894469112e-16im

 [-1,-1,-1] -> 16/5, 4  $\frac{4\chi_2}{\chi_1}(4/5,1)$
 [-1,-2,10] -> -16, 20
 [-1,-0.5,-1] -> -64, 80

In [306]:
4χ[2]/χ[1]*[4/5;1]

2-element Vector{Float64}:
 1.6
 2.0

In [307]:
function coc(par_f)
    return 4par_f[3]/par_f[2]*[-4/5 ; 1]
end

coc (generic function with 1 method)

In [308]:
coc([-1,-1,-10])

2-element Vector{Float64}:
 -32.0
  40.0

In [309]:
coc([-1,-0.5,-10])

2-element Vector{Float64}:
 -64.0
  80.0

Alternativamente podemos quedarnos con las expresiones en derivadas del los campos del fluido.
Es decir:

$(\zeta^a, \zeta^{ab})$ -> $f^j = (\mu, v, x_i)$ -> $(-1,0,0,0,0)$

$$
\partial_t c^j = \partial_i F^{ij}(f) + I^j(f) = \frac{\partial F^{ij}}{\partial f^k}\partial_i f^k + I^j(j)
$$

In [453]:
flu=[1.;0.0;0;0;0];
#con=ones(5)
par_f = [-1.,1.,-10.]
#par_f = [-1,-0.5,-10]
par_s = [-1/15π, 1/1π, -0.1/1π]
con = F(flu,zeros(5), par_f);
PP = inv(Jac(flu,con,par_f))*(1.0im*JFac(flu,par_f) + 1.0.*JSac(flu,par_f,par_s));

In [454]:
eigvals(PP)

5-element Vector{ComplexF64}:
 -0.01935884507765262 - 0.6366154008404264im
 -0.01935884507764971 + 0.6366154008404309im
 0.034862515028231934 + 0.6638069562731905im
 0.034862515028235154 - 0.6638069562731943im
  0.15657547918056527 + 6.586061503065444e-17im

In [318]:
Et = eigvecs(PP);

In [319]:
Ett = inv(Et);

In [320]:
H = adjoint(Ett)*Ett
#latexify(H)

5×5 Matrix{Float64}:
  0.513538     -3.61978e-14    0.00191108   -0.00501518   0.000363106
 -3.61978e-14  18.9672         0.0130682    -1.9499       0.00248296
  0.00191108    0.0130682     25.2733      -66.9076       1.15838
 -0.00501518   -1.9499       -66.9076      241.672       -6.22351
  0.000363106   0.00248296     1.15838      -6.22351      0.563921

In [322]:
H*PP - adjoint(PP)*H

5×5 Matrix{Float64}:
  0.0          5.55112e-17  -2.1684e-19   -5.20417e-16  -2.1684e-19
 -5.55112e-17  0.0          -6.59195e-17  -5.55112e-15  -2.64545e-17
  2.1684e-19   6.59195e-17   0.0           9.52127e-13  -3.9968e-15
  5.20417e-16  5.55112e-15  -9.52127e-13   0.0           1.33227e-13
  2.1684e-19   2.64545e-17   3.9968e-15   -1.33227e-13   0.0

In [323]:
eigvals(H)

5-element Vector{Float64}:
   0.35244445125213175
   0.5135378382406123
   6.294856102952448
  18.9739429431992
 260.8553589833234

In [326]:
eigvals(H*(1.0im.* JFac(flu,par_f) - 1.0.*JSac(flu,con,par_s)))

5-element Vector{ComplexF64}:
   -33.223638974522565 - 19.202303415731336im
 -4.596503152414722e-7 - 1.640027859765601im
    0.4330322848704424 + 7523.657188524685im
     91.47161178094568 - 28.33531128586182im
   2.205890516412259e6 - 253617.05026002906im

In [325]:
H*JSac(flu,con,p)

5×5 Matrix{Num}:
 0.0  0.0      0.163764(p₂^-1)      -1.08193(p₁^-1)     -0.100274(p₃^-1)
 0.0  0.0     43.6063(p₂^-1)      -292.215(p₁^-1)      -27.5938(p₃^-1)
 0.0  0.0   2101.66(p₂^-1)      -13856.2(p₁^-1)      -1277.42(p₃^-1)
 0.0  0.0  -7062.09(p₂^-1)       46745.5(p₁^-1)       4337.69(p₃^-1)
 0.0  0.0    177.139(p₂^-1)      -1179.98(p₁^-1)      -110.866(p₃^-1)

In [89]:
1/1.333333

0.7500001875000468

In [64]:
200/27

7.407407407407407